In [2]:
#import firebase_firestore
import numpy as np
from numpy.linalg import norm
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [4]:
#Initialize credentials to database
cred = None
def firestoreConnection():
    global cred
    cred = credentials.Certificate("mood-swing-6c9d0-firebase-adminsdk-9cm02-66f39cc0dd.json")
    firebase_admin.initialize_app(cred)

In [5]:
#CONSTANTS
#may need to be used if mood_index is passed instead of the mood itself
moods = ['sad','angry','energetic','excited','happy','content','calm','depressed'] 

In [29]:
#Function to store centroid values and the 5 closest songs to that centroid

#Parameters: user, dict of song names and their list of metadata values, the mood for the centroid to retrieve
def closestSongs(user_id, songs, mood):
    if cred == None:
        firestoreConnection()
    centroid = retrieveCentroid(user_id, mood)
    if len(songs) <= 5: return songs.keys()
    distances = []
    for (name, score) in songs.items():
        calculated_distance = cosineSimilarity(centroid, score)
        distances.append((name,calculated_distance))
    #sort the distances by value
    distances = sorted(distances, key=lambda x: x[1])
    print(distances[:5])
    print([pair[0] for pair in distances[:5]])
    #return the song names of the 5 smallest distances
    return [pair[0] for pair in distances[:5]]

def cosineSimilarity(arr1, arr2):
    return np.dot(arr1, arr2)/(norm(arr1)*norm(arr2))
        
def retrieveCentroid(user_id, mood):
    # Get a reference to the users collection
    users_ref = firestore.client().collection("users")
    # Get a reference to the specific user's document
    user_doc_ref = users_ref.document(user_id)
    # Get a reference to the "mood" document in the centroids subcollection
    mood_doc_ref = user_doc_ref.collection("centroids").document(mood) 
    return list(mood_doc_ref.get().to_dict().values()) #Centroid

In [30]:
import random
user_id = "0Lol0ym3tAahQZDjetO7C5i1VFJ3"
songs = {f"name{i}": [random.random() for j in range(13)] for i in range(10)}
mood = "angry"
song_list = closestSongs(user_id, songs, mood)

[('name0', 0.5928846502375861), ('name6', 0.6277662881590378), ('name3', 0.6503277463365955), ('name2', 0.6671520770405042), ('name1', 0.684620328809726)]
['name0', 'name6', 'name3', 'name2', 'name1']


In [ ]:
# def calculateCentroid(songs):
#     if len(songs) == 0: return None
#     #get scores for the mood
#     num_fields = len(songs[0][1]) #(- 1? if last field is label?)
#     num_songs = len(songs)
#     weight = 1/num_songs
#     cumulative_score = [0 for element in range(num_fields)]
#     for (_,scores) in songs:
#         cumulative_score = np.add(scores, cumulative_score)
#     return np.multiply(weight, cumulative_score) #Centroid